# Config

In [46]:
#editamos el path para poder accedera las funciones con la estructura actual de carpetas
from sys import path
import os
path.append(os.path.realpath('../'))

In [47]:
#importamos las funciones que hemos creado nosotros
from utils import functions as func

In [48]:
#importamos librería
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

import folium
import plotly.express as px
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [49]:
# Leemos los ficheros
ruta = '../data/'

df_calendar = pd.read_csv(ruta+'daily_calendar_with_events.csv')
df_sales = pd.read_csv(ruta+'item_sales.csv')
df_prices = pd.read_csv(ruta+'item_prices.csv')

# Datasets info

## Calendar

In [50]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


In [51]:
df_calendar.value_counts()

date        weekday    weekday_int  d       event         
2011-02-06  Sunday     2            d_9     SuperBowl         1
2011-08-01  Monday     3            d_185   Ramadan starts    1
2016-02-07  Sunday     2            d_1836  SuperBowl         1
2016-01-01  Friday     7            d_1799  NewYear           1
2015-11-26  Thursday   6            d_1763  Thanksgiving      1
2015-06-18  Thursday   6            d_1602  Ramadan starts    1
2015-04-05  Sunday     2            d_1528  Easter            1
2015-02-01  Sunday     2            d_1465  SuperBowl         1
2015-01-01  Thursday   6            d_1434  NewYear           1
2014-11-27  Thursday   6            d_1399  Thanksgiving      1
2014-06-29  Sunday     2            d_1248  Ramadan starts    1
2014-04-20  Sunday     2            d_1178  Easter            1
2014-02-02  Sunday     2            d_1101  SuperBowl         1
2014-01-01  Wednesday  5            d_1069  NewYear           1
2013-11-28  Thursday   6            d_1035  T

In [52]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         1913 non-null   object
 1   weekday      1913 non-null   object
 2   weekday_int  1913 non-null   int64 
 3   d            1913 non-null   object
 4   event        26 non-null     object
dtypes: int64(1), object(4)
memory usage: 74.9+ KB


In [53]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'], errors='coerce')

In [54]:
df_calendar.head()

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN


## Sales

In [55]:
df_sales.head()

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [56]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


Pasamos las columnas dia a fila

In [57]:
#Pasamos las columas d a filas par poder trabajar con ellas
# Cogemos las columnas.
columnas = df_sales.columns
list_id_vars = columnas[:7]   # son las columnas que necesitamos mantener.
list_value_vars = columnas[7:]

# Codigo para trasponer columnas a filas.
df_daily_sales = pd.melt(df_sales, id_vars=list_id_vars,
                   value_vars=list_value_vars ,
                   var_name='d', value_name='qty_sold')

In [58]:
df_daily_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 9 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   id          object
 1   item        object
 2   category    object
 3   department  object
 4   store       object
 5   store_code  object
 6   region      object
 7   d           object
 8   qty_sold    int64 
dtypes: int64(1), object(8)
memory usage: 3.9+ GB


## Prices

In [59]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [60]:
df_prices.head()

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858


In [61]:
df_prices.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [62]:
df_prices=df_prices.dropna(subset=["yearweek"])
df_prices.isnull().sum()

item          0
category      0
store_code    0
yearweek      0
sell_price    0
dtype: int64

In [63]:
# Pasamos la variable yearweek a modo fecha:
df_prices["yearweek"] = df_prices["yearweek"].astype("int32")

In [64]:
# Para hacer merge con el df_prices creamos columna d que es item+store_code dentro de prices
df_prices["id"]= df_prices["item"]+"_"+df_prices['store_code']

In [65]:
df_prices.head()

,item,category,store_code,yearweek,sell_price,id
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328,12.7414,ACCESORIES_1_001_NYC_1
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329,12.7414,ACCESORIES_1_001_NYC_1
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330,10.9858,ACCESORIES_1_001_NYC_1
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331,10.9858,ACCESORIES_1_001_NYC_1
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332,10.9858,ACCESORIES_1_001_NYC_1


In [66]:
del df_prices["item"], df_prices['store_code'], df_prices['category']

### Imputacion nulos en sellprice

# Union datasets

Realizamos la union del df_calendar y el df_daily sales a través de la variable d.

In [67]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         1913 non-null   datetime64[ns]
 1   weekday      1913 non-null   object        
 2   weekday_int  1913 non-null   int64         
 3   d            1913 non-null   object        
 4   event        26 non-null     object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 74.9+ KB


In [69]:
df_calendar_sales = pd.merge(df_daily_sales,df_calendar[['date','d']], on='d', how='inner')

Aplicamos resample para obtener las ventas por "semana". Se observa que del 2016 tenemos pocos meses, igual que 2011. Ojo hemos convertido "date" en índice.

In [ ]:
df_calendar_sales.head()

,id,item,category,department,store,store_code,region,d,qty_sold,date,weekday,weekday_int,event,week,month,year,yearweek
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,1,2011,201104
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,1,2011,201104
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,1,2011,201104
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,1,2011,201104
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0,2011-01-29,Saturday,1,NaN,4,1,2011,201104


In [70]:
df_calendar_sales.shape

(58327370, 10)

In [71]:
df_calendar_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58327370 entries, 0 to 58327369
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          object        
 1   item        object        
 2   category    object        
 3   department  object        
 4   store       object        
 5   store_code  object        
 6   region      object        
 7   d           object        
 8   qty_sold    int64         
 9   date        datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(8)
memory usage: 4.3+ GB


In [73]:
#df_calendar_sales.set_index("date").resample("W")["qty_sold"].sum() 

MemoryError: Unable to allocate 445. MiB for an array with shape (1, 58327370) and data type int64

In [74]:
# Cargar el DataFrame en lotes
chunk_size = 10000  # Puedes ajustar el tamaño del lote según tus necesidades
chunks = [df_calendar_sales[i:i + chunk_size] for i in range(0, len(df_calendar_sales), chunk_size)]

# Inicializar un DataFrame vacío para almacenar los resultados
result_df = pd.DataFrame()

# Iterar sobre los lotes y realizar el re-muestreo
for chunk in chunks:
    resampled_chunk = chunk.set_index("date").resample("W")["qty_sold"].sum()
    result_df = pd.concat([result_df, resampled_chunk])

# Restablecer el índice si lo deseas
result_df = result_df.reset_index()

In [75]:
result_df

,index,0
0,2011-01-30,12813.0
1,2011-01-30,8594.0
2,2011-01-30,9926.0
3,2011-01-30,13526.0
4,2011-01-30,8833.0
...,...,...
6101,2016-04-24,13721.0
6102,2016-04-24,18308.0
6103,2016-04-24,17948.0
6104,2016-04-24,13166.0


In [ ]:
df_calendar_sales

In [ ]:
df_merged_final= pd.merge(df_calendar_sales, df_prices, on=['id', "yearweek"], how='left')  # how puede ser 'inner', 'outer', 'left' o 'right'


In [ ]:
df_merged_final.info()

In [ ]:
df_merged_final.to_csv((ruta+'df_merged.csv'), index=False)

KeyboardInterrupt: 